In [32]:
# @title Параметры расчета
# Суммы:
Ostatok = 1_000_000.0
Vznos = 1_000.0
Admin = 2500.0
# Пол и возраст:
gender, age = 'M', 45
# % Доходностей:
brutto = 0.0
inflation = 0.042
# % Вознаграждений от СЧА
VF_pct, UK_pct, SD_pct, ASV_pct = 0.005 , 0.001, 0.003, 0.000125
# % Вознаграждений от ИД
VF_inv_pct, UK_inv_pct = 0.15, 0.03
system = 'new'
# Вероятности расторжения, смерти и фактор дожития
lapce, death = 0.005, 0.0
# Прочее
Pension_type = 'ДИ'
Tarif = 11 * 12
Pens_Term = 10 * 12
# Расчётные параметры:
DoR = (60 - age if gender == 'M' else 55 - age)*12 + 11
DoTO = DoR + Pens_Term if Pension_type in ['ДИ', 'СР'] else DoR if Pension_type == 'ЕВ' else 100*12

In [33]:
# @title Импорт библиотек и данных
import os, sys, requests, warnings, pandas as pd, numpy as np
from datetime import datetime as dt, timedelta
from sqlalchemy import create_engine, text
from bs4 import BeautifulSoup
from tqdm import tqdm
np.set_printoptions(formatter={'float': lambda x: '{:,.2f}'.format(x).replace(',', ' ')}) # чтобы печатал красиво
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', ' ')) # чтобы печатал красиво
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/MyDrive/Модель ОПС и НПО/'
death_tab = pd.read_excel(file_path + 'Актуарная система ОПС.xlsx')
year_month = pd.read_excel(file_path + 'Год_месяц.xlsx')
sys.path.append(file_path)
import Create_excel as crex
death_props = death_tab.iloc[:, :5]
KBD_data = death_tab.iloc[:, 8:13]

Mounted at /content/drive


In [ ]:
DoR, DoTO

(191, 311)

In [34]:
# @title 1) Исходные данные таблицы смертности и данные ставок КБД
calc_table = death_props.query("`Age (Y)` >= @age and `Age (Y)` < 111").rename(columns = {'Age (Y)' :'Age'})
calc_table = calc_table[['Age', 'q_x (M)']] if age == 'M' else calc_table[['Age', 'q_x (F)']]
calc_table = calc_table.rename(columns = {calc_table.columns[-1] : 'Death'})
calc_table['Год'] = range(calc_table.shape[0])
calc_table['noterm'] = 1.0
calc_table = calc_table.merge(KBD_data[['Y','МСФО 17']].rename(columns = {'Y':'Год', 'МСФО 17':'brutto'}), how = 'left', on = 'Год')
calc_table['live'] = 1.0
for i in range(1, len(calc_table)):
    calc_table.loc[i, 'live'] = (1-calc_table.loc[i, 'Death'])*calc_table.loc[i-1, 'live']
calc_table['netto'] = calc_table['brutto']*(1-VF_inv_pct) - ASV_pct/12 - VF_pct/12 if system == 'new' else \
  (calc_table['brutto']*(1-UK_inv_pct) - VF_pct - UK_pct - SD_pct - ASV_pct)*(1-VF_inv_pct)
calc_table = calc_table[['Год', 'Age', 'noterm', 'Death', 'live', 'brutto', 'netto']]
calc_table['Discont factor'] = 1.0
# Помесячный пересчет таблицы
calc_table_month = year_month.merge(calc_table, on = 'Год', how = 'left')#.to_excel(file_path + "Тест.xlsx")
calc_table_month['Death'] /= 12
for i in range(1, len(calc_table_month)):
    calc_table_month.loc[i, 'live'] = (1-calc_table_month.loc[i, 'Death'])*calc_table_month.loc[i-1, 'live']
calc_table_month['brutto'] = calc_table_month['brutto'].apply(lambda x: (1+x)**(1/12) - 1)
calc_table_month['netto'] = calc_table_month['brutto']*(1-VF_inv_pct) - ASV_pct/12 - VF_pct/12 if system == 'new' else \
  (calc_table_month['brutto']*(1-UK_inv_pct) - VF_pct/12 - UK_pct/12 -SD_pct/12 - ASV_pct/12)*(1-VF_inv_pct)
calc_table_month.loc[:12, 'Discont factor'] = 1.0
for i in range(12, len(calc_table_month)):
  calc_table_month.loc[i, 'Discont factor'] = calc_table_month.loc[i - 1, 'Discont factor'] / (1 + calc_table_month.loc[i, 'brutto'])
calc_table = calc_table_month.reset_index(drop = True)
for i in range(12, len(calc_table)):
  calc_table.at[i, 'noterm'] = calc_table.at[i - 1, 'noterm']*(1-lapce/12)

In [ ]:
calc_table.loc[:20]

,Год,Месяц,Age,noterm,Death,live,brutto,netto,Discont factor
0,0,1,45.00,1.00,0.00,1.00,NaN,NaN,1.00
1,0,2,45.00,1.00,0.00,1.00,NaN,NaN,1.00
2,0,3,45.00,1.00,0.00,1.00,NaN,NaN,1.00
3,0,4,45.00,1.00,0.00,1.00,NaN,NaN,1.00
4,0,5,45.00,1.00,0.00,1.00,NaN,NaN,1.00
5,0,6,45.00,1.00,0.00,1.00,NaN,NaN,1.00
6,0,7,45.00,1.00,0.00,1.00,NaN,NaN,1.00
7,0,8,45.00,1.00,0.00,1.00,NaN,NaN,1.00
8,0,9,45.00,1.00,0.00,1.00,NaN,NaN,1.00
9,0,10,45.00,1.00,0.00,1.00,NaN,NaN,1.00


In [ ]:
# @title 2) Расчёты нулевого года
calc_table.at[11, 'Остаток EOP'] = Ostatok
# calc_table.head(13)

In [ ]:
# @title 3) Расчёты с 1 года до выхода на пенсию
for i in range(12, DoR + 1):
  calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
  calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
  calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ВФ пост prob'] = VF_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
  calc_table.at[i, 'УК упр prob'] = UK_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'СД prob'] = SD_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'АСВ prob'] = ASV_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
    calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

  calc_table.at[i, 'Взносы sum'] = Vznos/12
  calc_table.at[i, 'Взносы prob'] = calc_table.at[i, 'Взносы sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'Админ sum'] = Admin/12 * (1 + inflation)**(calc_table.at[i-12, 'Год'])
  calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']

  calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
  calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP']
  calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']*(1-lapce/2)
  calc_table.at[i, 'Выкупная sum'] = calc_table.at[i, 'Остаток MOP']
  calc_table.at[i, 'Выкупная prob'] = calc_table.at[i, 'Выкупная sum'] * lapce * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm'] * (1 - calc_table.at[i, 'Death']/2)
# Прочее
  for cl in ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob', 'ДИ sum', 'ДИ prob']:
    calc_table.at[i, cl] = 0.0
  calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'])

In [ ]:

DoR

191

In [ ]:
# calc_table.loc[DoR+1]

In [ ]:
# calc_table.loc[DoR-12]

In [ ]:
# @title 4) Расчёты в момент выхода на пенсию
calc_table.at[DoR, 'ЕВ sum'] = calc_table.at[DoR, 'Остаток MOP'] if Pension_type == 'ЕВ' else 0.0
calc_table.at[DoR, 'ЕВ prob'] = calc_table.at[DoR, 'ЕВ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
calc_table.at[DoR, 'ПЖ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Tarif if Pension_type == 'ПЖ' else 0.0
calc_table.at[DoR, 'ПЖ prob'] = calc_table.at[DoR, 'ПЖ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
calc_table.at[DoR, 'СР sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'СР' else 0.0
calc_table.at[DoR, 'СР prob'] = calc_table.at[DoR, 'СР sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
calc_table.at[DoR, 'ДИ sum'] = calc_table.at[DoR, 'Остаток MOP'] / Pens_Term if Pension_type == 'ДИ' else 0.0
calc_table.at[DoR, 'ДИ prob'] = calc_table.at[DoR, 'ДИ sum'] * calc_table.at[DoR, 'live'] * calc_table.at[DoR, 'noterm']
calc_table.at[DoR, 'Остаток EOP'] = max(0, calc_table.at[DoR, 'Остаток MOP'] - calc_table.loc[DoR, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())

In [ ]:
calc_table.loc[DoR, ['Остаток MOP', 'ДИ sum']]

,191
Остаток MOP,6 926 521.80
ДИ sum,57 721.02


In [ ]:
i = 16
Pens_Term - (calc_table.at[i, 'Год'] - DoR), DoR, Pens_Term

(np.int64(310), 191, 120)

In [ ]:
i = 17
calc_table.loc[i, 'ПЖ sum']
# calc_table.loc[i, ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob']].sum()

0.0

In [37]:
# @title 5) Расчёты после выхода на пенсию
for i in range(DoR + 1, DoTO):
  calc_table.at[i, 'noterm'] = calc_table.at[i - 1, 'noterm']
  calc_table.at[i, 'Остаток BOP'] = calc_table.at[i - 1, 'Остаток EOP']
  calc_table.at[i, 'ИД brutto sum'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'brutto']
  calc_table.at[i, 'ИД brutto prob'] = calc_table.at[i, 'ИД brutto sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ИД netto prob'] = calc_table.at[i, 'Остаток BOP'] * calc_table.at[i, 'netto'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ВФ пост prob'] = VF_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'УК усп prob'] = UK_inv_pct * calc_table.at[i, 'ИД brutto prob']
  calc_table.at[i, 'УК упр prob'] = UK_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'СД prob'] = SD_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'АСВ prob'] = ASV_pct/12 * calc_table.at[i, 'Остаток BOP'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'ВФ перем prob'] = VF_inv_pct * (calc_table.at[i, 'ИД brutto prob'] if system == 'new' else calc_table.at[i, 'ИД brutto prob'] - \
    calc_table.loc[i, ['ВФ пост prob', 'УК усп prob', 'УК упр prob', 'СД prob', 'АСВ prob']].sum())

  calc_table.at[i, 'Взносы sum'] = 0.0
  calc_table.at[i, 'Взносы prob'] = 0.0

  calc_table.at[i, 'Остаток MOP'] = calc_table.at[i, 'Остаток BOP'] * (1 + calc_table.at[i, 'netto']) + calc_table.at[i, 'Взносы sum']
  calc_table.at[i, 'Наследники sum'] = calc_table.at[i, 'Остаток MOP'] if Pension_type != 'ПЖ' else 0
  calc_table.at[i, 'Наследники prob'] = calc_table.at[i, 'Наследники sum'] * calc_table.at[i, 'Death'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']
  calc_table.at[i, 'Выкупная sum'] = 0.0
  calc_table.at[i, 'Выкупная prob'] = 0.0

  calc_table.at[i, 'ЕВ sum'] = 0.0
  calc_table.at[i, 'ЕВ prob'] = 0.0
  calc_table.at[i, 'ПЖ sum'] = calc_table.at[i - 1, 'ПЖ sum'] * (1 + calc_table.at[i, 'netto'])
  calc_table.at[i, 'ПЖ prob'] = calc_table.at[i, 'ПЖ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
  calc_table.at[i, 'СР sum'] = calc_table.at[i - 1, 'СР sum'] * (1 + calc_table.at[i, 'netto'])
  calc_table.at[i, 'СР prob'] = calc_table.at[i, 'СР sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
  calc_table.at[i, 'ДИ sum'] = calc_table.at[i, 'Остаток MOP'] / (Pens_Term - (calc_table.at[i, 'Месяц'] - DoR)) if Pension_type == 'ДИ' else 0.0
  calc_table.at[i, 'ДИ prob'] = calc_table.at[i, 'ДИ sum'] * calc_table.at[i, 'live'] * calc_table.at[i, 'noterm']
  calc_table.at[i, 'Остаток EOP'] = max(0, calc_table.at[i, 'Остаток MOP'] - calc_table.loc[i, ['ЕВ sum', 'ПЖ sum', 'СР sum', 'ДИ sum']].sum())
  calc_table.at[i, 'Админ sum'] = Admin/12 * (1 + inflation)**(calc_table.at[i-12, 'Год']) if calc_table.loc[i, ['ЕВ sum', 'ЕВ prob', 'ПЖ sum', 'ПЖ prob', 'СР sum', 'СР prob']].sum() > 0.01 else 0.0
  calc_table.at[i, 'Админ prob'] = calc_table.at[i, 'Админ sum'] * calc_table.at[i-1, 'live'] * calc_table.at[i-1, 'noterm']

KeyError: 'Остаток EOP'

In [36]:
calc_table.at[DoR, 'Год'], DoR

(np.int64(15), 191)

In [35]:
Pens_Term

120

In [ ]:
# @title 6) Расчёт summary и выгрузка в excel
EV_total = (calc_table['Discont factor'] * calc_table['ЕВ prob']).sum()
PZ_total = (calc_table['Discont factor'] * calc_table['ПЖ prob']).sum()
SR_total = (calc_table['Discont factor'] * calc_table['СР prob']).sum()
DI_total = (calc_table['Discont factor'] * calc_table['ДИ prob']).sum()

Nasled_total = (calc_table['Discont factor'] * calc_table['Наследники prob']).sum()
Vznos_total = (calc_table['Discont factor'] * calc_table['Взносы prob']).sum()
VF_total = (calc_table['Discont factor'] * (calc_table['ВФ пост prob'] + calc_table['ВФ перем prob'])).sum()
# Расчёты выкупной суммы:
temp_tab = calc_table.groupby('Год', as_index = False).agg({'Discont factor':'last', 'Выкупная prob':'last'})
Vykup_total = (temp_tab['Discont factor'] * temp_tab['Выкупная prob']).sum()

Admin_total = (calc_table['Discont factor'] * calc_table['Админ prob']).sum()
UK_total = (calc_table['Discont factor'] * (calc_table['УК усп prob'] + calc_table['УК упр prob'])).sum()
SD_total = (calc_table['Discont factor'] * calc_table['СД prob']).sum()
ASV_total = (calc_table['Discont factor'] * calc_table['АСВ prob']).sum()

Earned_total = Ostatok + Vznos_total - (EV_total + PZ_total + SR_total + DI_total) - (Nasled_total + Vykup_total) - (Admin_total + UK_total + SD_total + ASV_total)
Earned_total_2 = VF_total - Admin_total - (UK_total + SD_total) if system == 'new' else VF_total - Admin_total

Summary = pd.DataFrame({'Категория':['Пенсии']*4 + ['Выплаты']*4 + ['Расходы'] * 4 + ['Фин рез'] * 2,
        'Показатель':['ЕВ', 'ПЖ', 'СР', 'ДИ', 'Наслед', 'Выкупная', 'Взносы', 'ВФ', 'Админ', 'УК', 'СД', 'АСВ', 'Фонд заработал 1', 'Фонд заработал 2'],
        'Сумма':[EV_total, PZ_total, SR_total, DI_total, Nasled_total, Vykup_total, Vznos_total, VF_total, Admin_total, UK_total, SD_total, ASV_total, Earned_total, Earned_total_2]})

calc_table = calc_table.query("Год != 0")

In [ ]:
Summary

,Категория,Показатель,Сумма
0,Пенсии,ЕВ,0.00
1,Пенсии,ПЖ,898 349.67
2,Пенсии,СР,0.00
3,Пенсии,ДИ,0.00
4,Выплаты,Наслед,26 738.28
5,Выплаты,Выкупная,57 179.79
6,Выплаты,Взносы,5 354.58
7,Выплаты,ВФ,407 930.92
8,Расходы,Админ,19 219.17
9,Расходы,УК,81 586.18


In [ ]:
# Запись в excel
with pd.ExcelWriter(file_path + "НПО_Result.xlsx", engine = 'openpyxl') as writer:
    Summary.to_excel(writer, sheet_name = 'Свод итогов', index = False)
    calc_table.to_excel(writer, sheet_name = 'Потоки детально', index = False)
crex.format_output(writer, file_path + "НПО_Result.xlsx")

In [ ]:
pd.DataFrame(Summary.T.iloc[2:3, :], columns = list(Summary.T.iloc[1:2, :]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Сумма,0.00,0.00,0.00,80 022.44,7 453.71,0.00,50 890.88,47 799.68,15 373.65,10 284.27,5 117.57,213.23,32 426.03,32 426.03
